In [ ]:
!sudo apt update && sudo apt upgrade -y
!sudo apt-get install -y openjdk-11-jre
!sudo apt-get install -y openjdk-11-jdk
!pip install -q python-terrier
import pyterrier as pt
if not pt.started():
    pt.init()

Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian buster-updates InRelease [51.9 kB]
Get:3 http://security.debian.org/debian-security buster/updates InRelease [65.4 kB]
Get:4 http://deb.debian.org/debian buster/main amd64 Packages [7,906 kB]
Get:5 http://deb.debian.org/debian buster-updates/main amd64 Packages [8,792 B]
Get:6 http://security.debian.org/debian-security buster/updates/main amd64 Packages [315 kB]
Fetched 8,469 kB in 2s (4,071 kB/s)



9 packages can be upgraded. Run 'apt list --upgradable' to see them.




The following packages will be upgraded:
  icu-devtools libexpat1 libexpat1-dev libicu-dev libicu63 libsasl2-2
  libsasl2-modules-db libtiff5 tzdata
9 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
Need to get 18.6 MB of archives.
After this operation, 6,144 B of additional disk space will be used.
Get:1 http://security.debian.org/debian-security buster/updates/main amd64 libsasl2-modules-db amd64 2.1.27+dfsg-

In [ ]:
def msmarco_generate():
    dataset = pt.get_dataset("trec-deep-learning-passages")
    with pt.io.autoopen('collections/msmarco-passage/collection.tsv', 'rt') as corpusfile:
        for l in corpusfile:
            docno, passage = l.split("\t")
            yield {'docno' : docno, 'text' : passage}

In [ ]:
dataset = pt.get_dataset("trec-deep-learning-passages")

In [ ]:
iter_indexer = pt.IterDictIndexer("./passage_index")
indexref = iter_indexer.index(msmarco_generate(), meta={'docno' : 20, 'text': 4096})

index = pt.IndexFactory.of(indexref)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: specifying meta and meta_lengths in IterDictIndexer.index() is deprecated, use constructor instead
  
Exception in thread Thread-3:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/root/venv/lib/python3.7/site-packages/pyterrier/index.py", line 750, in _write_fifos
    for doc in it:
  File "/root/venv/lib/python3.7/site-packages/pyterrier/index.py", line 642, in _filter_iterable
    for doc in it:
  File "<ipython-input-1-c5ec92a7765e>", line 3, in msmarco_generate
    with pt.io.autoopen('collections/msmarco-passage/collection.tsv', 'rt') as corpusfile:
  File "/root/venv/lib/python3.7/site-packages/pyterrier/io.py", line 29, in autoopen
    return open(filename, mode)
FileNotFoundError

JavaException: JVM exception occurred: No IndexLoaders were supported for indexref ./passage_index/data.properties; It may be your ref has the wrong location. Alternatively, Terrier is misconfigured - did you import the correct package to deal with this indexref? java.lang.UnsupportedOperationException

In [ ]:
tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
bm25b = pt.BatchRetrieve(index, controls={"wmodel":"BM25"})
bm25_qe = pt.BatchRetrieve(index, wmodel="BM25", controls={"qe":"on", "qemodel" : "Bo1"})
# pl2 = pt.BatchRetrieve(index, wmodel="PL2")

18:15:56.505 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.
18:15:57.739 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.
18:15:59.073 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.
18:16:00.350 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.


In [ ]:
result = pt.Experiment([bm25], 
                       dataset.get_topics("test-2019"), 
                       dataset.get_qrels("test-2019"),
                       eval_metrics=["recip_rank", "ndcg_cut_10","map"])
print(result)

       name  recip_rank  ndcg_cut_10       map
0  BR(BM25)    0.795028      0.47954  0.370004


In [ ]:
result_qe= pt.Experiment([bm25_qe], 
                       dataset.get_topics("test-2019"), 
                       dataset.get_qrels("test-2019"),
                       eval_metrics=["recip_rank", "ndcg_cut_10","map"])
print(result_qe)

       name  recip_rank  ndcg_cut_10       map
0  BR(BM25)    0.788124      0.50862  0.399897


In [ ]:
result_tf = pt.Experiment([tf_idf], 
                       dataset.get_topics("test-2019"), 
                       dataset.get_qrels("test-2019"),
                       eval_metrics=["recip_rank", "ndcg_cut_10","map"])
print(result_tf)

         name  recip_rank  ndcg_cut_10       map
0  BR(TF_IDF)    0.795028      0.47831  0.369486


In [ ]:
result_per_query = pt.Experiment([bm25], 
                                 dataset.get_topics("test-2019"),
                                 dataset.get_qrels("test-2019"), 
                                 eval_metrics=["recip_rank", "ndcg_cut_10","map"],
                                 perquery=True)

/usr/local/lib/python3.8/dist-packages/pyterrier/pipelines.py:108: UserWarning: 157 topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.
  warn(f'{backfill_count} topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.')


In [ ]:
result_per_query = result_per_query[result_per_query["value"].notnull()]
result_per_query[result_per_query["measure"]=='map'].sort_values(by="value")

,name,qid,measure,value
96,BR(BM25),1121709,map,0.000239
6,BR(BM25),1063750,map,0.003193
93,BR(BM25),443396,map,0.005157
12,BR(BM25),489204,map,0.055242
39,BR(BM25),962179,map,0.058701
24,BR(BM25),527433,map,0.064043
78,BR(BM25),1106007,map,0.094742
27,BR(BM25),1037798,map,0.109533
48,BR(BM25),451602,map,0.122208
66,BR(BM25),833860,map,0.139674


Query Expansion

In [ ]:
Bo1 = pt.rewrite.Bo1QueryExpansion(index)
KL = pt.rewrite.KLQueryExpansion(index)
# RM3 = pt.rewrite.RM3(index)



18:18:41.421 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.
18:18:42.366 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.


In [ ]:
pt.Experiment(
    [
            bm25, 
            bm25 >> Bo1 >> bm25, 
            bm25 >> KL >> bm25, 
            
    ],
    dataset.get_topics('test-2019'),
    dataset.get_qrels("test-2019"),
    eval_metrics=["recip_rank", "ndcg_cut_10","map"],
    names=["BM25", "+Bo1", "+KL"]
    )

,name,recip_rank,ndcg_cut_10,map
0,BM25,0.795028,0.479540,0.370004
1,+Bo1,0.788124,0.508620,0.399897
2,+KL,0.785023,0.505715,0.397658


In [ ]:
result_per_query_Bo = pt.Experiment([bm25 >> Bo1 >> bm25], 
                                 dataset.get_topics("test-2019"),
                                 dataset.get_qrels("test-2019"), 
                                 eval_metrics=["recip_rank", "ndcg_cut_10","map"],
                                 perquery=True)

/usr/local/lib/python3.8/dist-packages/pyterrier/pipelines.py:108: UserWarning: 157 topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.
  warn(f'{backfill_count} topic(s) not found in qrels. Scores for these topics are given as NaN and should not contribute to averages.')


In [ ]:
result_per_query_Bo = result_per_query_Bo[result_per_query_Bo["value"].notnull()]
result_per_query_Bo[result_per_query_Bo["measure"]=='map'].sort_values(by="value")

,name,qid,measure,value
39,"Compose(Compose(BR(BM25), <pyterrier.rewrite.B...",1121709,map,0.000158
6,"Compose(Compose(BR(BM25), <pyterrier.rewrite.B...",1063750,map,0.002752
90,"Compose(Compose(BR(BM25), <pyterrier.rewrite.B...",443396,map,0.004997
12,"Compose(Compose(BR(BM25), <pyterrier.rewrite.B...",1106007,map,0.019114
105,"Compose(Compose(BR(BM25), <pyterrier.rewrite.B...",527433,map,0.038237
126,"Compose(Compose(BR(BM25), <pyterrier.rewrite.B...",962179,map,0.052588
99,"Compose(Compose(BR(BM25), <pyterrier.rewrite.B...",489204,map,0.062827
93,"Compose(Compose(BR(BM25), <pyterrier.rewrite.B...",451602,map,0.126669
15,"Compose(Compose(BR(BM25), <pyterrier.rewrite.B...",1110199,map,0.135263
0,"Compose(Compose(BR(BM25), <pyterrier.rewrite.B...",1037798,map,0.138890


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=eeb7fcb4-84cb-4f4a-9830-9d1fcbc47e1d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>